In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "Virny":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/Virny


# Single Run Single Model Interface Usage

In this example, we are going to audit 1 model for stability and fairness, visualize metrics, and create an analysis report. We will use `compute_model_metrics_with_config` interface that will conduct the auditing pipeline for this model. For that, we will need to do next steps:

* Initialize input variables

* Compute subgroup metrics

* Make group metrics composition

* Create metrics visualizations and an analysis report

## Import dependencies

In [4]:
import os
import pandas as pd
from datetime import datetime, timezone

from sklearn.tree import DecisionTreeClassifier

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from virny.user_interfaces.metrics_computation_interfaces import compute_model_metrics_with_config
from virny.utils.custom_initializers import create_config_obj, read_model_metric_dfs
from virny.custom_classes.metrics_visualizer import MetricsVisualizer
from virny.custom_classes.metrics_composer import MetricsComposer
from virny.datasets.base import BaseDataLoader
from virny.preprocessing.basic_preprocessing import preprocess_dataset

## Initialize Input Variables

Based on the library flow, we need to create 3 input objects for a user interface:

* A **dataset class** that is a wrapper above the user’s raw dataset that includes its descriptive attributes like a target column, numerical columns, categorical columns, etc. This class must be inherited from the BaseDataset class, which was created for user convenience.

* A **config yaml** that is a file with configuration parameters for different user interfaces for metrics computation.

* Finally, a **models config** that is a Python dictionary, where keys are model names and values are initialized models for analysis. This dictionary helps conduct audits of multiple models for one or multiple runs and analyze different types of models.

In [5]:
TEST_SET_FRACTION = 0.2
DATASET_SPLIT_SEED = 42

### Create a Dataset class

Based on the BaseDataset class, your **dataset class** should include the following attributes:

* **Obligatory attributes**: dataset, target, features, numerical_columns, categorical_columns

* **Optional attributes**: X_data, y_data, columns_with_nulls

For more details, please refer to the library documentation.

In [6]:
class CompasDataset(BaseDataLoader):
    """
    Dataset class for COMPAS dataset that contains sensitive attributes among feature columns.

    Parameters
    ----------
    subsample_size
        Subsample size to create based on the input dataset

    """
    def __init__(self, dataset_path, subsample_size=None):
        df = pd.read_csv(dataset_path)
        if subsample_size:
            df = df.sample(subsample_size)

        int_columns = ['recidivism', 'age', 'age_cat_25 - 45', 'age_cat_Greater than 45',
                       'age_cat_Less than 25', 'c_charge_degree_F', 'c_charge_degree_M', 'sex']
        int_columns_dct = {col: "int" for col in int_columns}
        df = df.astype(int_columns_dct)

        target = 'recidivism'
        numerical_columns = ['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count']
        categorical_columns = ['race', 'age_cat_25 - 45', 'age_cat_Greater than 45',
                               'age_cat_Less than 25', 'c_charge_degree_F', 'c_charge_degree_M', 'sex']

        super().__init__(
            full_df=df,
            target=target,
            numerical_columns=numerical_columns,
            categorical_columns=categorical_columns,
        )

In [7]:
data_loader = CompasDataset(dataset_path=os.path.join('virny', 'datasets', 'COMPAS.csv'))
data_loader.X_data[data_loader.X_data.columns[:5]].head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count
0,25,0.0,-2.340451,1.0,-15.010999
1,26,0.0,0.000000,0.0,0.000000
2,21,0.0,0.000000,0.0,0.000000
3,29,0.0,0.000000,0.0,6.000000
4,40,0.0,0.000000,0.0,7.513697


In [8]:
column_transformer = ColumnTransformer(transformers=[
    ('categorical_features', OneHotEncoder(handle_unknown='ignore', sparse=False), data_loader.categorical_columns),
    ('numerical_features', StandardScaler(), data_loader.numerical_columns),
])

In [9]:
base_flow_dataset = preprocess_dataset(data_loader, column_transformer, TEST_SET_FRACTION, DATASET_SPLIT_SEED)

### Create a config object

`compute_model_metrics_with_config` interface requires that your **yaml file** includes the following parameters:

* **dataset_name**: a name of your dataset; it will be used to name files with metrics.

* **bootstrap_fraction**: the fraction from a train set in the range [0.0 - 1.0] to fit models in bootstrap (usually more than 0.5).

* **n_estimators**: the number of estimators for bootstrap to compute subgroup variance metrics.

* **sensitive_attributes_dct**: a dictionary where keys are sensitive attribute names (including attribute intersections), and values are privileged values for these attributes. Currently, the library supports only intersections among two sensitive attributes. Intersectional attributes must include '&' between sensitive attributes. You do not need to specify privileged values for intersectional groups since they will be derived from privileged values in sensitive_attributes_dct for each separate sensitive attribute in this intersectional pair.


In [10]:
ROOT_DIR = os.path.join('docs', 'examples')
config_yaml_path = os.path.join(ROOT_DIR, 'experiment_config.yaml')
config_yaml_content = """
dataset_name: COMPAS
bootstrap_fraction: 0.8
n_estimators: 100
sensitive_attributes_dct: {'sex': 0, 'age': 25, 'race': 'Caucasian', 'sex&race': None}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [11]:
config = create_config_obj(config_yaml_path=config_yaml_path)
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results',
                                     f'{config.dataset_name}_Metrics_{datetime.now(timezone.utc).strftime("%Y%m%d__%H%M%S")}')

### Create a models config

**models_config** is a Python dictionary, where keys are model names and values are initialized models for analysis

In [12]:
models_config = {
    'DecisionTreeClassifier': DecisionTreeClassifier(criterion='gini',
                                                     max_depth=20,
                                                     max_features=0.6,
                                                     min_samples_split=0.1),
}

## Subgroup Metrics Computation

After the variables are input to a user interface, the interface creates a **generic pipeline** based on the input dataset class to hide preprocessing complexity and provide handy attributes and methods for different types of model analysis. Later this generic pipeline is used in subgroup analyzers that compute different sets of metrics. As for now, our library supports **Subgroup Variance Analyzer** and **Subgroup Error Analyzer**, but it is easily extensible to any other analyzers. When the variance and error analyzers complete metrics computation, their metrics are combined, returned in a matrix format, and stored in a file if defined.

In [13]:
metrics_df = compute_model_metrics_with_config(models_config['DecisionTreeClassifier'], 'DecisionTreeClassifier', base_flow_dataset,
                                               config, SAVE_RESULTS_DIR_PATH, save_results=True, verbose=0)

Model seed:  990




2023-03-19 19:34:44 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

2023-03-19 19:34:44 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-03-19 19:34:48 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics


Look at several columns in top rows of computed metrics

In [14]:
metrics_df[metrics_df.columns[:6]].head(20)

,Metric,overall,sex_priv,sex_dis,age_priv,age_dis
0,Mean,0.524368,0.624285,0.499418,0.491800,0.525919
1,Std,0.091029,0.103845,0.087828,0.085703,0.091282
2,IQR,0.115913,0.131572,0.112003,0.112554,0.116073
3,Entropy,0.231701,0.227051,0.232862,0.000000,0.235781
4,Jitter,0.147724,0.142311,0.149075,0.094373,0.150264
5,Per_Sample_Accuracy,0.678693,0.692559,0.675231,0.683542,0.678462
6,Label_Stability,0.792727,0.805592,0.789515,0.845417,0.790218
7,TPR,0.636943,0.360000,0.689394,0.625000,0.637584
8,TNR,0.743590,0.897059,0.697105,0.666667,0.746881
9,PPV,0.666667,0.658537,0.667482,0.652174,0.667447


## Group Metrics Composition

**Metrics Composer** is responsible for this second stage of the model audit. Currently, it computes our custom group fairness and variance metrics, but extending it for new group metrics is very simple. We noticed that more and more group metrics have appeared during the last decade, but most of them are based on the same subgroup metrics. Hence, such a separation of subgroup and group metrics computation allows one to experiment with different combinations of subgroup metrics and avoid subgroup metrics recomputation for a new set of grouped metrics.

In [15]:
models_metrics_dct = read_model_metric_dfs(SAVE_RESULTS_DIR_PATH, model_names=['DecisionTreeClassifier'])

In [16]:
metrics_composer = MetricsComposer(models_metrics_dct, config.sensitive_attributes_dct)

Compute composed metrics

In [17]:
models_composed_metrics_df = metrics_composer.compose_metrics()

In [18]:
models_composed_metrics_df.head(20)

,Metric,sex,age,race,Model_Name
0,Equalized_Odds_TPR,0.329394,0.012584,0.263353,DecisionTreeClassifier
1,Equalized_Odds_FPR,0.199954,-0.080214,0.147869,DecisionTreeClassifier
2,Disparate_Impact,1.889320,0.996790,1.337232,DecisionTreeClassifier
3,Statistical_Parity_Difference,0.486162,-0.003076,0.261527,DecisionTreeClassifier
4,Accuracy_Parity,-0.012670,0.052579,0.004583,DecisionTreeClassifier
5,Label_Stability_Ratio,0.980042,0.934709,1.011960,DecisionTreeClassifier
6,IQR_Parity,-0.019569,0.003519,-0.003671,DecisionTreeClassifier
7,Std_Parity,-0.016017,0.005579,-0.000531,DecisionTreeClassifier
8,Std_Ratio,0.845760,1.065100,0.994188,DecisionTreeClassifier
9,Jitter_Parity,0.006764,0.055891,-0.006394,DecisionTreeClassifier


## Metrics Visualization

**Metrics Visualizer** provides metrics visualization and reporting functionality. It unifies different preprocessing methods for result metrics and creates various data formats required for visualizations. Hence, users can simply call methods of the Metrics Visualizer class and get custom plots for diverse metrics analysis.

In [19]:
visualizer = MetricsVisualizer(models_metrics_dct, models_composed_metrics_df, config.dataset_name,
                               model_names=['DecisionTreeClassifier'],
                               sensitive_attributes_dct=config.sensitive_attributes_dct)

In [20]:
visualizer.create_overall_metrics_bar_char(
    metrics_names=['TPR', 'PPV', 'Accuracy', 'F1', 'Selection-Rate', 'Positive-Rate'],
    metrics_title="Error Metrics"
)

alt.Chart(...)

In [21]:
visualizer.create_overall_metrics_bar_char(
    metrics_names=['Label_Stability'],
    reversed_metrics_names=['Std', 'IQR', 'Jitter'],
    metrics_title="Variance Metrics"
)

alt.Chart(...)